In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("skill_prediction - prediction (1).csv")
df.head(2)

,Job description /job requirements,COLING,GALAXC,True Labels,GALAXC-best,fasttext
0,[CLS] requirements technical academic educatio...,"['Project Management', 'SQL', 'Integration', '...","['Product Development', 'Manufacturing', 'Nego...","['Management', 'Microsoft Office', 'Cloud Comp...","['Product Development', 'Manufacturing', 'Cros...","['Management', 'Manufacturing', 'Marketing', '..."
1,[CLS] requirements apply s ww bluechipcareers ...,"['Financial Analysis', 'Management', 'Portfoli...","['Customer Service', 'Risk Management', 'Sales...","['Sales', 'Investments', 'Negotiation', 'Selli...","['Management', 'Risk Management', 'Mergers & A...","['Customer Service', 'Management', 'Microsoft ..."


In [3]:
# Uncomment the following code only if we want to append predictions with latest outputs...

with open('../data/COLING/models/GraphXMLModelNR_finetuned-fasttext/predictions_20','r') as f:
    data = f.readlines()
data_pred = [(x.split('\t')[0]) for x in data]
df['latest'] = data_pred
df.to_csv("skill_prediction - prediction (1).csv", index=None)


In [4]:
df.head()

,Job description /job requirements,COLING,GALAXC,True Labels,GALAXC-best,fasttext,latest
0,[CLS] requirements technical academic educatio...,"['Project Management', 'SQL', 'Integration', '...","['Product Development', 'Manufacturing', 'Nego...","['Management', 'Microsoft Office', 'Cloud Comp...","['Product Development', 'Manufacturing', 'Cros...","['Management', 'Manufacturing', 'Marketing', '...","['Management', 'Project Management', 'Leadersh..."
1,[CLS] requirements apply s ww bluechipcareers ...,"['Financial Analysis', 'Management', 'Portfoli...","['Customer Service', 'Risk Management', 'Sales...","['Sales', 'Investments', 'Negotiation', 'Selli...","['Management', 'Risk Management', 'Mergers & A...","['Customer Service', 'Management', 'Microsoft ...","['Management', 'Leadership', 'Financial Analys..."
2,[CLS] requirementsdiploma related field strong...,"['Java', 'Software Development', 'JavaScript',...","['C++', 'Java', 'Software Development', 'Testi...","['Agile Methodologies', 'Java', 'Software Deve...","['MySQL', 'Agile Methodologies', 'SQL', 'C#', ...","['XML', 'SQL', 'Java', 'Agile Methodologies', ...","['Java', 'Software Development', 'C++', 'XML',..."
3,[CLS] requirementspossible activities achieve ...,"['Management', 'Business Development', 'Market...","['Engineering', 'Product Development', 'Manufa...","['Leadership', 'Management', 'Project Manageme...","['Product Development', 'Engineering', 'Projec...","['Project Management', 'Marketing', 'Marketing...","['Manufacturing', 'Project Management', 'Manag..."
4,[CLS] requirementsjob requirements minimum gce...,"['Brand Development', 'Digital Marketing', 'Ma...","['Digital Media', 'Digital Strategy', 'Digital...","['Social Media', 'Advertising', 'Concept Devel...","['Creative Strategy', 'Digital Media', 'Advert...","['Advertising', 'Creative Direction', 'Copywri...","['Advertising', 'Photoshop', 'Social Media', '..."


In [5]:
true= df['True Labels'].apply( lambda x: eval(x))
true = true.tolist()
coling = df['COLING'].apply( lambda x: eval(x))

coling = coling.tolist()
jd = df['Job description /job requirements'].tolist()

Ga = df['GALAXC'].apply( lambda x: eval(x))
GAL = df['GALAXC-best'].apply( lambda x: eval(x))
fs = df['fasttext'].apply(lambda x: eval(x))
lt = df['latest'].apply(lambda x: eval(x))
lt.tolist()
fs.tolist()
Ga = Ga.tolist()
GAL = GAL.tolist()


In [6]:
for item in fs:
    assert len(item) ==20

In [7]:
class Metric:
    def __init__(self):
        self.epsilon = 1.0e-4
        pass

    def __call__(self, outputs, target):
        raise NotImplementedError

    def reset(self):
        raise NotImplementedError

    def value(self):
        raise NotImplementedError

    def name(self):
        raise NotImplementedError

class Accuracy(Metric):
    def __init__(self,topK):
        super(Accuracy,self).__init__()
        self.topK = topK
        self.reset()

    def __call__(self, logits, target):
        _, pred = logits.topk(self.topK, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        self.correct_k = correct[:self.topK].view(-1).float().sum(0)
        self.total = target.size(0)

    def reset(self):
        self.correct_k = 0
        self.total = 0

    def value(self):
        return float(self.correct_k)  / self.total

    def name(self):
        return 'accuracy'


class AccuracyThresh(Metric):
    def __init__(self,thresh = 0.5):
        super(AccuracyThresh,self).__init__()
        self.thresh = thresh
        self.reset()

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid()
        self.y_true = target

    def reset(self):
        self.correct_k = 0
        self.total = 0

    def value(self):
        data_size = self.y_pred.size(0)
        acc = np.mean(((self.y_pred>self.thresh).type(torch.DoubleTensor)==self.y_true.type(torch.DoubleTensor)).float().cpu().numpy(), axis=1).sum()
        return acc / data_size

    def name(self):
        return 'accuracy'

class MRR(Metric):
    def __init__(self):
        super(MRR,self).__init__()
        self.rr = []

    def __call__(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true

        for i in range(len(self.y_pred)):
            y_t = np.where(self.y_true[i]==1)[0]
            y_t_len = len(y_t)
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0)
            for i in range(len(y_p_index)):
                if y_p_index[i] in y_t:
                    self.rr.append(1/float(i+1))
                    break

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.rr))*100

    def name(self):
        return 'mrr'

    def show(self):
        print("MRR: {}".format(self.value()))

class NDCG(Metric):
    def __init__(self):
        super(NDCG,self).__init__()
        self.NDCG = []
        self.kvals = [5,10,30,50,100]
        self.reset()

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid().cpu().numpy()
        self.y_true = target.cpu().numpy()
        
        for i in range(len(self.y_pred)):
            y_t = np.where(self.y_true[i]==1)[0]
            y_t_len = len(y_t)
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0)
            idcg = np.sum([1.0/np.log2(x+2) for x in range(y_t_len)])
            ndcg = []
            for k in self.kvals:
                dcg = 0 
                for i in range(0,k):
                    if y_p_index[i] in y_t:
                        dcg = dcg + 1.0/np.log2(i+2)
                ndcg.append(dcg/idcg)
            self.NDCG.append(ndcg)

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.NDCG),axis=0)*100

    def name(self):
        return 'ndcg'

    def show(self):
        idx = 0
        result = self.value()
        for k in self.kvals:
            print("NDCG@{}: {}".format(k,result[idx]))
            idx += 1
        return result

class Recall(Metric):
    def __init__(self):
        super(Recall,self).__init__()
        self.Recall = []
        self.kvals = [5,10,20,30,50,100]
        self.reset()

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid().cpu().numpy()        
        self.y_true = target.cpu().numpy()
        for i in range(len(self.y_pred)):
            recall = []
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0) # What is this doing, needs to sort highest probability
            y_t = np.where(self.y_true[i]==1)[0] # Returns index where condition true. an array 
            y_t_len = len(y_t)

            for k in self.kvals:
                correct = len(np.intersect1d(y_t,y_p_index[0:k]))
                recall.append(correct/(y_t_len+self.epsilon))
            self.Recall.append(recall)
        

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.Recall),axis=0)*100

    def name(self):
        return 'recall'
    
    def show(self):
        idx = 0
        result = self.value()
        for k in self.kvals:
            print("Recall@{}: {}".format(k,result[idx]))
            idx += 1
        return result




class Precision(Metric):
    def __init__(self):
        super(Precision,self).__init__()
        self.Precision = []
        self.kvals = [5,10,20,30,50,100]
        self.reset()
        self.count_ind=[]

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid().cpu().numpy()       
        self.y_true = target.cpu().numpy()
        for i in range(len(self.y_pred)):
            precision = []
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0) # index where predicted values are true
            y_t = np.where(self.y_true[i]==1)[0] # indexes where actual truth labels 
            y_t_len = len(y_t)  # how many true labels

            # print("Number of skills actually correct :",y_t_len)
            # print("Number of skills we predict are correct for k 5,10,30,50,100:")

            for k in self.kvals:
                correct = len(np.intersect1d(y_t,y_p_index[0:k]))
                # print(correct)
                precision.append(correct/(k+self.epsilon))
            self.Precision.append(precision)
            # print("Indexes matched",np.intersect1d(y_t,y_p_index[0:k]))
        

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.Precision),axis=0)*100

    def name(self):
        return 'precision'
    
    def show(self):
        idx = 0
        result = self.value()
        for k in self.kvals:
            print("Precision@{}: {}".format(k,result[idx]))
            idx += 1
        return result
# implicit metrics are evaluated based on the top-20 predections 

class Implicit_Metrics(Metric):

    def __init__(self):
        super(Implicit_Metrics,self).__init__()
    
    def read_skills(self, path):
        skill_list = []
        with open(path, 'r') as f:
            for line in f.readlines():
                skill = line.replace('\n','')
                skill_list.append(skill)
        return skill_list

    def present_skills(self, job_description, skill_list):
        skills = []
        for skill in skill_list:
            if (' ' + skill.lower().strip() + ' ') in (' ' + job_description.lower().strip() + ' '):
                if skill not in skills:
                    skills.append(skill)
        return skills

    def generate_sentence(self, input_ids):
        input_ids = input_ids.tolist()
        sentence = ""
        for idx in input_ids:
            if self.i2w[idx][:2]=="##":
                sentence += self.i2w[idx][2:]
            else:
                sentence += " " + self.i2w[idx]
        return sentence.strip()
    
    def job_labels(self, label_indices):
        labels = []
        for idx in label_indices:
            labels.append(self.i2l[idx])
        return labels

    def intersection(self, lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3

class EIM(Implicit_Metrics):
    def __init__(self):
        super(EIM,self).__init__()
        self.EIM = []
        self.reset()

    def __call__(self, jd, y_pred, y_true):
        self.jd = jd
        self.y_pred = y_pred       
        self.y_true = y_true
        for i in range(len(self.y_pred)):
            self.EIM.append(self.eim(self.jd[i], self.y_pred[i], self.y_true[i]))
        
            
    def eim(self, jd, output, target):

        f = open('skill_list (1).txt').readlines()
        skill_lis = [x.split('\n')[0] for x in f]
#         print(skill_lis)
        
        explicit_skills = self.present_skills(jd, skill_lis) # skill in jd against all skills 

        if len(self.intersection(target, explicit_skills))==0:
            return 1
        return len(self.intersection(output, explicit_skills)) / (len(self.intersection(target, explicit_skills)))


    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.EIM))*100

    def name(self):
        return 'eim'
    
    def show(self):
        result = self.value()
        print("EIM: {}".format(result))

class RIIM(Implicit_Metrics):
    def __init__(self):
        super(RIIM,self).__init__()
        self.RIIM = []
        self.reset()

    def __call__(self, jd, y_pred, y_true):
        self.jd = jd
        self.y_pred = y_pred       
        self.y_true = y_true
        for i in range(len(self.y_pred)):
            self.RIIM.append(self.riim(self.jd[i], self.y_pred[i], self.y_true[i]))
        
    def riim(self, jd, output, target):
        f = open('skill_list (1).txt').readlines()
        skill_lis = [x.split('\n')[0] for x in f]
        
        explicit_skills = self.present_skills(jd, skill_lis)
        implicit_skills = [val for val in target if val not in explicit_skills]
        predicted_implicit_skills = [val for val in output if val in implicit_skills]

        return len(predicted_implicit_skills)/(len(implicit_skills) + self.epsilon)

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.RIIM))*100

    def name(self):
        return 'riim'
    
    def show(self):
        result = self.value()
        print("RIIM: {}".format(result))

class REIM(Implicit_Metrics):
    def __init__(self):
        super(REIM,self).__init__()
        self.REIM = []
        self.reset()

    def __call__(self, jd, y_pred, y_true):
        self.jd = jd
        self.y_pred = y_pred       
        self.y_true = y_true
        for i in range(len(self.y_pred)):
            self.REIM.append(self.reim(self.jd[i], self.y_pred[i], self.y_true[i]))
        
            
    def reim(self, jd, output, target):

        f = open('skill_list (1).txt').readlines()
        skill_lis = [x.split('\n')[0] for x in f]
#         print(skill_lis)
        
        explicit_skills = self.present_skills(jd, skill_lis) # skill in jd against all skills 
        
        return len(self.intersection(output, explicit_skills)) / (len(explicit_skills)+self.epsilon)


    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.REIM))*100

    def name(self):
        return 'reim'
    
    def show(self):
        result = self.value()
        print("REIM: {}".format(result))


class AUC(Metric):
    '''
    AUC score
    micro:
            Calculate metrics globally by considering each element of the label
            indicator matrix as a label.
    macro:
            Calculate metrics for each label, and find their unweighted
            mean.  This does not take label imbalance into account.
    weighted:
            Calculate metrics for each label, and find their average, weighted
            by support (the number of true instances for each label).
    samples:
            Calculate metrics for each instance, and find their average.
    Example:
        >>> metric = AUC(**)
        >>> for epoch in range(epochs):
        >>>     metric.reset()
        >>>     for batch in batchs:
        >>>         logits = model()
        >>>         metric(logits,target)
        >>>         print(metric.name(),metric.value())
    '''

    def __init__(self,task_type = 'binary',average = 'binary'):
        super(AUC, self).__init__()

        assert task_type in ['binary','multiclass']
        assert average in ['binary','micro', 'macro', 'samples', 'weighted']

        self.task_type = task_type
        self.average = average

    def __call__(self,logits,target):
        if self.task_type == 'binary':
            self.y_prob = logits.sigmoid().data.cpu().numpy()
        else:
            self.y_prob = logits.softmax(-1).data.cpu().detach().numpy()
        self.y_true = target.cpu().numpy()

    def reset(self):
        self.y_prob = 0
        self.y_true = 0

    def value(self):
        auc = roc_auc_score(y_score=self.y_prob, y_true=self.y_true, average=self.average)
        return auc

    def name(self):
        return 'auc'

class F1Score(Metric):
    '''
    F1 Score
    binary:
            Only report results for the class specified by ``pos_label``.
            This is applicable only if targets (``y_{true,pred}``) are binary.
    micro:
            Calculate metrics globally by considering each element of the label
            indicator matrix as a label.
    macro:
            Calculate metrics for each label, and find their unweighted
            mean.  This does not take label imbalance into account.
    weighted:
            Calculate metrics for each label, and find their average, weighted
            by support (the number of true instances for each label).
    samples:
            Calculate metrics for each instance, and find their average.
    Example:
        >>> metric = F1Score(**)
        >>> for epoch in range(epochs):
        >>>     metric.reset()
        >>>     for batch in batchs:
        >>>         logits = model()
        >>>         metric(logits,target)
        >>>         print(metric.name(),metric.value())
    '''
    def __init__(self,thresh = 0.5, normalizate = True,task_type = 'multiclass',average = 'micro',search_thresh = False):
        super(F1Score).__init__()
        assert task_type in ['binary','multiclass']
        assert average in ['binary','micro', 'macro', 'samples', 'weighted']

        self.thresh = thresh
        self.task_type = task_type
        self.normalizate  = normalizate
        self.search_thresh = search_thresh
        self.average = average

    def thresh_search(self,y_prob):
        best_threshold = 0
        best_score = 0
        for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
            self.y_pred = y_prob > threshold
            score = self.value()
            if score > best_score:
                best_threshold = threshold
                best_score = score
        return best_threshold,best_score

    def __call__(self,logits,target):
        self.y_true = target.cpu().numpy()
        if self.normalizate and self.task_type == 'binary':
            y_prob = logits.sigmoid().data.cpu().numpy()
        elif self.normalizate and self.task_type == 'multiclass':
            y_prob = logits.softmax(-1).data.cpu().detach().numpy()
        else:
            y_prob = logits.cpu().detach().numpy()

        if self.task_type == 'binary':
            if self.thresh and self.search_thresh == False:
                self.y_pred = (y_prob > self.thresh ).astype(int)
                self.value()
            else:
                thresh,f1 = self.thresh_search(y_prob = y_prob)
                print(f"Best thresh: {thresh:.4f} - F1 Score: {f1:.4f}")

        if self.task_type == 'multiclass':
            self.y_pred = np.argmax(y_prob, 1)

    def reset(self):
        self.y_pred = 0
        self.y_true = 0

    def value(self):
        f1 = f1_score(y_true=self.y_true, y_pred=self.y_pred, average=self.average)
        return f1

    def name(self):
        return 'f1'
    
    def show(self):
        result = self.value()
        print("F1: {}".format(result))
      
        

class ClassReport(Metric):
    '''
    class report
    '''
    def __init__(self,target_names = None):
        super(ClassReport).__init__()
        self.target_names = target_names

    def reset(self):
        self.y_pred = 0
        self.y_true = 0

    def value(self):
        score = classification_report(y_true = self.y_true,
                                      y_pred = self.y_pred,
                                      target_names=self.target_names)
        print(f"\n\n classification report: {score}")

    def __call__(self,logits,target):
        _, y_pred = torch.max(logits.data, 1)
        self.y_pred = y_pred.cpu().numpy()
        self.y_true = target.cpu().numpy()

    def name(self):
        return "class_report"

class MultiLabelReport(Metric):
    '''
    multi label report
    '''
    def __init__(self,id2label = None):
        super(MultiLabelReport).__init__()
        self.id2label = id2label

    def reset(self):
        self.y_prob = 0
        self.y_true = 0

    def __call__(self,logits,target):

        self.y_prob = logits.sigmoid().data.cpu().detach().numpy()
        self.y_true = target.cpu().numpy()

    def value(self):
        for i, label in self.id2label.items():
            auc = roc_auc_score(y_score=self.y_prob[:, i], y_true=self.y_true[:, i])
            print(f"label:{label} - auc: {auc:.4f}")

    def name(self):
        return "multilabel_report"


In [8]:
## latest Metrics
x = EIM()
x(jd,lt,true)
y = REIM()
y(jd,lt,true)
z = RIIM()
z(jd,lt,true)

In [9]:
x.show()
y.show()
z.show()

EIM: 102.89201954061488
REIM: 21.112104106333952
RIIM: 37.15310874462651


In [40]:
## Coling Metrics 
x = EIM()
x(jd,coling,true)
y = REIM()
y(jd,coling,true)
z = RIIM()
z(jd,coling,true)
x.show()
y.show()
z.show()

EIM: 115.88814640810699
REIM: 25.730109648276965
RIIM: 64.6082733676545


In [41]:
# GalaXC MiniLM 
x = EIM()
x(jd,GAL,true)
y = REIM()
y(jd,GAL,true)
z = RIIM()
z(jd,GAL,true)
x.show()
y.show()
z.show()

EIM: 86.44667650951534
REIM: 17.768765714071684
RIIM: 27.278701915258978


## Let's analyse what happens with frequent skills in all predictions

In [59]:
f = open('skill_list (1).txt').readlines()
skill_lis = [x.split('\n')[0] for x in f]

In [60]:
from collections import defaultdict as ddict

dicti = ddict(int)
for item in true:
    for j in item:
        dicti[j]+=1
        
skill_lis_fil =[]
for key,val in dicti.items():
    if dicti[key]>20: # frequency should be more than 20
        skill_lis_fil.append(key)

    

In [61]:

lis_counts=[]

for i, (x,y) in enumerate(zip(true,GAL)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts.append(count)
# print(lis_counts)

In [62]:
df['num_frequent_skills_predicted_galaxc_minilm'] = lis_counts

In [63]:
df.head()

,Job description /job requirements,COLING,GALAXC,True Labels,GALAXC-best,fasttext,num_frequent_skills_predicted_galaxc_minilm
0,[CLS] requirements technical academic educatio...,"['Project Management', 'SQL', 'Integration', '...","['Product Development', 'Manufacturing', 'Nego...","['Management', 'Microsoft Office', 'Cloud Comp...","['Product Development', 'Manufacturing', 'Cros...","['Management', 'Manufacturing', 'Marketing', '...",1
1,[CLS] requirements apply s ww bluechipcareers ...,"['Financial Analysis', 'Management', 'Portfoli...","['Customer Service', 'Risk Management', 'Sales...","['Sales', 'Investments', 'Negotiation', 'Selli...","['Management', 'Risk Management', 'Mergers & A...","['Customer Service', 'Management', 'Microsoft ...",2
2,[CLS] requirementsdiploma related field strong...,"['Java', 'Software Development', 'JavaScript',...","['C++', 'Java', 'Software Development', 'Testi...","['Agile Methodologies', 'Java', 'Software Deve...","['MySQL', 'Agile Methodologies', 'SQL', 'C#', ...","['XML', 'SQL', 'Java', 'Agile Methodologies', ...",12
3,[CLS] requirementspossible activities achieve ...,"['Management', 'Business Development', 'Market...","['Engineering', 'Product Development', 'Manufa...","['Leadership', 'Management', 'Project Manageme...","['Product Development', 'Engineering', 'Projec...","['Project Management', 'Marketing', 'Marketing...",11
4,[CLS] requirementsjob requirements minimum gce...,"['Brand Development', 'Digital Marketing', 'Ma...","['Digital Media', 'Digital Strategy', 'Digital...","['Social Media', 'Advertising', 'Concept Devel...","['Creative Strategy', 'Digital Media', 'Advert...","['Advertising', 'Creative Direction', 'Copywri...",11


In [64]:
lis_counts_coling=[]

for i, (x,y) in enumerate(zip(true,coling)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts_coling.append(count)
# print(lis_counts)

In [65]:
df['num_frequent_skills_predicted_coling'] = lis_counts_coling

In [66]:
lis_counts_fs=[]

for i, (x,y) in enumerate(zip(true,fs)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts_fs.append(count)
# print(lis_counts)

In [67]:
df['num_frequent_skills_predicted_fs'] = lis_counts_fs

In [68]:
df.head(20)

,Job description /job requirements,COLING,GALAXC,True Labels,GALAXC-best,fasttext,num_frequent_skills_predicted_galaxc_minilm,num_frequent_skills_predicted_coling,num_frequent_skills_predicted_fs
0,[CLS] requirements technical academic educatio...,"['Project Management', 'SQL', 'Integration', '...","['Product Development', 'Manufacturing', 'Nego...","['Management', 'Microsoft Office', 'Cloud Comp...","['Product Development', 'Manufacturing', 'Cros...","['Management', 'Manufacturing', 'Marketing', '...",1,12,5
1,[CLS] requirements apply s ww bluechipcareers ...,"['Financial Analysis', 'Management', 'Portfoli...","['Customer Service', 'Risk Management', 'Sales...","['Sales', 'Investments', 'Negotiation', 'Selli...","['Management', 'Risk Management', 'Mergers & A...","['Customer Service', 'Management', 'Microsoft ...",2,2,2
2,[CLS] requirementsdiploma related field strong...,"['Java', 'Software Development', 'JavaScript',...","['C++', 'Java', 'Software Development', 'Testi...","['Agile Methodologies', 'Java', 'Software Deve...","['MySQL', 'Agile Methodologies', 'SQL', 'C#', ...","['XML', 'SQL', 'Java', 'Agile Methodologies', ...",12,19,19
3,[CLS] requirementspossible activities achieve ...,"['Management', 'Business Development', 'Market...","['Engineering', 'Product Development', 'Manufa...","['Leadership', 'Management', 'Project Manageme...","['Product Development', 'Engineering', 'Projec...","['Project Management', 'Marketing', 'Marketing...",11,19,12
4,[CLS] requirementsjob requirements minimum gce...,"['Brand Development', 'Digital Marketing', 'Ma...","['Digital Media', 'Digital Strategy', 'Digital...","['Social Media', 'Advertising', 'Concept Devel...","['Creative Strategy', 'Digital Media', 'Advert...","['Advertising', 'Creative Direction', 'Copywri...",11,14,13
5,[CLS] requirementsrecruit select train assign ...,"['Management', 'Leadership', 'Strategic Planni...","['Customer Service', 'Management', 'Sales', 'L...","['Customer Service', 'Leadership', 'Management...","['Customer Service', 'Microsoft Office', 'Lead...","['Management', 'Customer Service', 'Training',...",10,16,9
6,[CLS] requirements course work understanding v...,"['Testing', 'Engineering', 'Matlab', 'C', 'Pro...","['OOP', 'Video Games', 'FPGA', 'Demand Plannin...","['Manufacturing', 'Testing', 'Unix', 'C', 'Sem...","['Manufacturing', 'FDA', 'CMOS', 'Testing', 'A...","['Testing', 'Java', 'Simulations', 'Quality As...",2,4,4
7,[CLS] requirements1 university degree equivale...,"['Project Management', 'Management', 'Windows ...","['Manufacturing', 'Testing', 'Process Improvem...","['Customer Service', 'Leadership', 'Management...","['Integration', 'Cloud Computing', 'Engineerin...","['Linux', 'Java', 'Testing', 'Manufacturing', ...",1,5,1
8,[CLS] requirementsrequirements proven experien...,"['Customer Service', 'Microsoft Excel', 'Micro...","['Human Resources', 'Performance Management', ...","['Customer Service', 'Event Planning', 'Human ...","['Human Resources', 'Management', 'Talent Mana...","['Microsoft Excel', 'Customer Service', 'Micro...",7,15,15
9,[CLS] requirements diploma degree life science...,"['Clinical Trials', 'Clinical Development', 'R...","['Biotechnology', 'FDA', 'Pharmaceutical Indus...","['Clinical Research', 'Biotechnology', 'Clinic...","['FDA', 'Biotechnology', 'Clinical Trials', 'G...","['Research', 'Mental Health Counseling', 'Publ...",2,2,0


In [71]:
counts_fs_coling =0
for i,num in enumerate(lis_counts_fs):
    if num > lis_counts_coling[i]:
        counts_fs_coling+=num-lis_counts_coling[i]
print(counts_fs_coling)

536
